In [1]:
!pip install selenium pandas
!pip install webdriver-manager
!pip install bs4
!pip install html5lib
!pip install mysql-connector-python

In [2]:
# imports go here
import mysql.connector
from bs4 import BeautifulSoup
import html5lib
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [3]:
# update chrome driver and run
path = '/Users/rohitpoks/Downloads/chromedriver'
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/rohitpoks/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [4]:
link = 'https://dish.avifoodsystems.com/oberlincollege/111/182/week' # breakfast
# link = 'https://dish.avifoodsystems.com/oberlincollege/111/183/week' # lunch
# link = 'https://dish.avifoodsystems.com/oberlincollege/111/184/week' # dinner

In [5]:
# get page source
def openPage():
    driver_ = webdriver.Chrome(service=s)
    driver_.maximize_window()
    driver_.get(link)
    time.sleep(5)
    return driver_.page_source

In [6]:
# get driver
def openPageReturnDriver():
    driver_ = webdriver.Chrome(service=s)
    driver_.maximize_window()
    driver_.get(link)
    time.sleep(5)
    return driver_

In [7]:
html_source = openPage()
soup = BeautifulSoup(html_source, 'html.parser')  # run parser on page source

In [8]:
# wait for 10 seconds, let it open

In [9]:
lunchDict = {}
day = 0
changeDay = False
itemCounter = 1
currentStationValue = 0
for para in soup.find_all("div",  {"class": ["station-header", "product-name pl-2", "text-center text-ming avi-text"]}):
    if (para.get_text().strip()[0:6] == "Sunday" or para.get_text()[0:6] == "Monday" or para.get_text()[0:7] == "Tuesday" or para.get_text()[0:9] == "Wednesday" or para.get_text()[0:8] == "Thursday" or para.get_text()[0:6] == "Friday" or para.get_text()[0:8] == "Saturday"):
            changeDay = True
    elif para.get_text().isupper() or para.get_text() == "Vegan Breakfast":
        if changeDay:
            currentStationValue = 0
        currentStationValue+=1
        itemCounter = 1
        currentStation = para.get_text()
    else:
        itemCounter+=1
        currentString = para.get_text()
        if changeDay:
            day+=1
            changeDay = False
        if not para.get_text() in lunchDict:
            lunchDict[para.get_text()] = [day, currentStationValue, itemCounter]

In [10]:
# extract from db object
# get from database and store in hashset
class GetNames(object):
    def __init__(self):
        self.create_connection()
        #self.process_item(array)
        #self.get_item_name(setObject)
        
    def create_connection(self):
        self.conn = mysql.connector.connect(
            host='35.232.237.179',
            user='avifresher',
            passwd='avifreshsucks42069',
            database='avifresher'
        )
        self.curr = self.conn.cursor()

    def get_item_name(self, setObject):
        self.curr.execute("""SELECT Name from stevenson""")
        setObject = self.curr.fetchall()
        return setObject
        

In [11]:
newObject = GetNames()
mySet = {}
mySet = newObject.get_item_name(mySet)

In [12]:
# add items to db
def getSpacesAndNumbers(x):
    result = ""
    for i in range(0, len(x)):
        if ((ord(x[i]) == 32) or (ord(x[i]) >= 48 and ord(x[i]) <= 57)):
            result += x[i]
    return result



def addAttributesToList(array, x, count):
    innerCount = 0
    for i in range(len(x)):
        if (ord(x[i]) >= 48 and ord(x[i]) <= 57):
            array[count][innerCount + 15] = array[count][innerCount + 15] * 10 + int(x[i])
        else:
            if (ord(x[i - 1]) >= 48 and ord(x[i-1]) <= 57):
                innerCount+=1
                
def addAllergensToList(array, x, count):
    if (x[-3:] == "Soy"):
        array[count][1] = 1
    if (x[-5:] == "Dairy"):
        array[count][2] = 1
    if (x[-5:] == "Wheat"):
        array[count][3] = 1
    if (x == "Vegetarian"):
        array[count][4] = 1
    if (x == "Vegan"):
        array[count][5] = 1
    if (x[-3:] == "Egg"):
        array[count][6] = 1
    if (x[-4:] == "Fish"):
        array[count][7] = 1
    if (x[-6:] == "Peanut"):
        array[count][8] = 1
    if (x[-6:] == "Sesame"):
        array[count][9] = 1
    if (x[-9:] == "Shellfish"):
        array[count][10] = 1
    if (x[-7:] == "TreeNut"):
        array[count][11] = 1
    if (x == "GlutenSensitive"):
        array[count][12] = 1
    if (x == "Halal"):
        array[count][13] = 1
    if (x == "Kosher"):
        array[count][14] = 1
        
count = 0;
rows, cols = (1000, 1000)
array = [ [0]*1000 for i in range(1000)]
doIt = True
for para in soup.find_all("div",  {"class": ["station-header", "product-name pl-2"]}):
    temp = (para.get_text(),)
    if not para.get_text().isupper() and doIt and para.get_text() != "" and not (temp in mySet) and para.get_text() != "Vegan Breakfast":
        dictReturns = lunchDict[para.get_text()]
        xpath =  '//*[@id="bg-blur"]/app-root/ng-component/mat-sidenav-container/mat-sidenav-content/div/app-menu-item-week/div/div/div[' + str(dictReturns[0]) + ']/div[' + str(dictReturns[1]) + ']/div[' + str(dictReturns[2]) + ']/div[1]'
        time.sleep(10)
        array[count][0] = para.get_text()
        print(array[count][0], " attributes:")
        driver_ = openPageReturnDriver()
        x = driver_.find_element(By.XPATH, xpath).click()
        currentItemSource = driver_.page_source
        soup = BeautifulSoup(currentItemSource, 'html.parser')  # run parser on page source
        for innerPara in soup.find_all("p", {"class": "text-ming text-center avi-text"}):
            x = innerPara.get_text()
            print(x)
            x = x.replace(" ", "")
            addAllergensToList(array, x, count)
        for innerPara in soup.find_all("table", {"class": "performance-facts__table"}):
            print("inner para is " + innerPara.get_text())
            temp = getSpacesAndNumbers(innerPara.get_text())
            addAttributesToList(array, temp, count)
            print(array[count])
        count+=1
        driver_.close()


Sausage Link, Pork  attributes:
 Gluten Sensitive 
inner para is  Amount Per Serving Calories 263 % Daily Value*Total Fat 26g  Saturated Fat 9g Trans Fat 0g Cholesterol 44mg Sodium 551mg Total Carbohydrate 0g  Dietary Fiber 0g  Total Sugar 0g  Added Sugar 0g Protein 6g 
['Sausage Link, Pork', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 263, 26, 9, 0, 44, 551, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Blueberry Buttermilk Pancakes  attributes:
 Dairy 
 Wheat 
 Vegetarian 
inner para is  Amount Per Serving Calories 294 % Daily Value*Total Fat 3g  Saturated Fat 1g Trans Fat 0g Cholesterol 0mg Sodium 887mg Total Carbohydrate 59g  Dietary Fiber 2g  Total Sugar 10g  Added Sugar 7g Protein 9g 
['Blueberry Buttermilk Pancakes', 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 294, 3, 1, 0, 0, 887, 59, 2, 10, 7, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
# insert unique food items into db
class MyTableClass(object):
    def __init__(self, array):
        self.create_connection()
        #self.create_table()
        self.process_item(array)
        
    def create_connection(self):
        self.conn = mysql.connector.connect(
            host='35.232.237.179',
            user='avifresher',
            passwd='avifreshsucks42069',
            database='avifresher'
        )
        self.curr = self.conn.cursor()

    def process_item(self, array):
        count = 0;
        while (array[count][0] != 0):
                print(array[count][0])
                self.curr.execute("""insert ignore into stevenson values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", (array[count][0], array[count][1], array[count][2], array[count][3], array[count][4], array[count][5], array[count][6], array[count][7], array[count][8], array[count][9], array[count][10], array[count][11], array[count][12], array[count][13], array[count][14], array[count][15], array[count][16], array[count][17], array[count][18], array[count][19], array[count][20], array[count][21], array[count][22], array[count][23], array[count][24], array[count][25], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
                self.conn.commit()
                count+=1
                
    def reset(self):
        self.curr.execute("""UPDATE stevenson SET monday_breakfast = NULL, monday_lunch = NULL, monday_dinner = null, tuesday_breakfast = NULL, tuesday_lunch = null, tuesday_dinner = null, wednesday_breakfast = NULL, wednesday_lunch = NULL, wednesday_dinner = NULL, thursday_breakfast = NULL, thursday_lunch = NULL, thursday_dinner = NULL, friday_breakfast = NULL, friday_lunch = NULL, friday_dinner = NULL, saturday_breakfast = NULL, saturday_lunch = NULL, saturday_dinner = Null, sunday_breakfast = NULL, sunday_lunch = NULL, sunday_dinner = NULL, station = NULL;""")
        self.conn.commit()

In [14]:
x = MyTableClass(array)
x.reset()

Sausage Link, Pork
Home Fries
Sunrise Southwest Breakfast Burrito
Blueberry Buttermilk Pancakes
Quesadilla Sausage Egg and Cheese


In [15]:
# *******************************************************************************
# run for breakfast, lunch and dinner first and then use the reset function below

In [16]:
def openPage(link):
    driver_ = webdriver.Chrome(service=s)
    driver_.maximize_window()
    driver_.get(link)
    time.sleep(5)
    return driver_.page_source

In [17]:
#html_source = driver.page_source
html_source = openPage('https://dish.avifoodsystems.com/oberlincollege/111/182/week')
soup = BeautifulSoup(html_source, 'html.parser')  # run parser on page source

In [18]:
class forUpdate(object):
    def __init__(self, number):
        self.create_connection()
        
    def create_connection(self):
        self.conn = mysql.connector.connect(
            host='35.232.237.179',
            user='avifresher',
            passwd='avifreshsucks42069',
            database='avifresher'
        )
        self.curr = self.conn.cursor()

    def update(self, meal_number):
        day = 0
        getName = ['', 'sunday', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday']
        meal = ''
        if meal_number == 0:
            meal = '_breakfast'
        elif meal_number == 1:
            meal = '_lunch'
        else:
            meal = '_dinner'
            
        currentStation = ''
        for para in soup.find_all("div",  {"class": ["station-header", "product-name pl-2", "text-center text-ming avi-text"]}):
            if (para.get_text().strip()[0:6] == "Sunday" or para.get_text()[0:6] == "Monday" or para.get_text()[0:7] == "Tuesday" or para.get_text()[0:9] == "Wednesday" or para.get_text()[0:8] == "Thursday" or para.get_text()[0:6] == "Friday" or para.get_text()[0:8] == "Saturday"):
                    day += 1
            elif para.get_text().isupper() or para.get_text() == "Vegan Breakfast":
                assign = para.get_text()
                if para.get_text()[-5:].lower() == "lunch":
                    assign = para.get_text()[0: len(para.get_text()) - 5]
                if para.get_text()[-6:].lower() == "dinner":
                    assign = para.get_text()[0: len(para.get_text()) - 6]
                currentStation = assign.upper()
            elif (para.get_text() != ""):
                currentString = para.get_text()
                column_name = getName[day] + meal
                query = "UPDATE stevenson set " + "".join(column_name) + " = 1, station = %s where Name = %s"
                parameters = (currentStation, para.get_text())
                self.curr.execute(query, parameters)
                self.conn.commit()
    

In [19]:
x = forUpdate(0)
x.update(0)

In [20]:

html_source = openPage('https://dish.avifoodsystems.com/oberlincollege/111/183/week')
soup = BeautifulSoup(html_source, 'html.parser')  # run parser on page source

In [21]:
# wait for 10
x.update(1)

In [22]:
html_source = openPage('https://dish.avifoodsystems.com/oberlincollege/111/184/week')
soup = BeautifulSoup(html_source, 'html.parser')  # run parser on page source

In [23]:
# wait for 10
x.update(2)